In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Batch (all rows): seamless/global OK + IDW fallback for chlor_a.

- Reads SUMMARY_CSV (with column 'file', optional 'in-situ chl-a').
- For each scene:
  * Loads chlor_a (+ l2_flags LAND), computes seamless/global OK (log10) with IDW fallback.
  * Saves PNGs, GeoTIFFs (if rectilinear + rasterio), and _seamless.nc copy with bands.
  * Appends a row to a CSV with diagnostics: center value, 3x3/5x5/10x10 stats, etc.

All outputs go to a single OUTPUT_DIR (created if missing).
"""

from pathlib import Path, PureWindowsPath
import os, re, json, shutil
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from math import cos
from datetime import datetime

# ---------- Optional deps ----------
try:
    from pykrige.ok import OrdinaryKriging as PKOrdinaryKriging
    HAVE_PYKRIGE = True
except Exception:
    HAVE_PYKRIGE = False

try:
    from scipy.spatial import cKDTree
    HAVE_KDTREE = True
except Exception:
    HAVE_KDTREE = False

try:
    from scipy.ndimage import zoom as scipy_zoom
    HAVE_SCIPY_NDIMAGE = True
except Exception:
    HAVE_SCIPY_NDIMAGE = False

try:
    import rasterio
    from rasterio.transform import Affine
    HAVE_RASTERIO = True
except Exception:
    HAVE_RASTERIO = False

try:
    from netCDF4 import Dataset
    HAVE_NETCDF4 = True
except Exception:
    HAVE_NETCDF4 = False


# ============================== SETTINGS ==============================
SITE        = "LISCO"
BASE_DIR    = Path("/Volumes/purkislab2a/") / "Mingyue" / SITE
SUMMARY_CSV = BASE_DIR / f"{SITE}_l2_summary_v2.csv"

# All outputs go here
OUTPUT_DIR  = BASE_DIR / "krige_outputs_3"   # <— change if you like

# Map Windows drive(s) to macOS mount(s)
DRIVE_MAP   = {"Y:": Path("/Volumes/purkislab2a")}

# L2 variable
GROUP       = "geophysical_data"
VARNAME     = "chlor_a"

# Diagnostics target (used if CSV lacks nearest_row/col)
TARGET_LAT  = 40.954517
TARGET_LON  = -73.341767

# Seamless/global-OK knobs (same as old script)
N_CLOSEST_POINTS   = 50
VARIOGRAM_MODEL    = "exponential"
RANDOM_SEED        = 123
MIN_RADIUS_M       = 40_000.0
MAX_GLOBAL_OBS     = 120_000
BATCH_TARGETS      = 120_000

# PNG look
CHL_VMIN, CHL_VMAX = 0.0, 5.0
DPI                = 160

# Verbosity
VERBOSE = True


# ============================== HELPERS ==============================
def log(msg: str):
    if VERBOSE:
        ts = datetime.now().strftime("%H:%M:%S")
        print(f"[{ts}] {msg}")

def remap_windows_path_to_local(p: str) -> Path:
    """Map 'Y:\\foo\\bar' to '/Volumes/purkislab2a/foo/bar' on macOS."""
    if p is None:
        return Path("")
    p = str(p).strip().strip('"').strip("'")
    if not p:
        return Path("")
    if os.name == "nt":
        return Path(p)
    if re.match(r"^[A-Za-z]:\\", p):
        wp = PureWindowsPath(p)
        base = DRIVE_MAP.get(wp.drive)
        if base is not None:
            return base.joinpath(*wp.parts[1:])
    return Path(p)

def ensure_output_dir() -> Path:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    return OUTPUT_DIR

def latlon_to_xy_m(lat, lon, lat0=None):
    if lat0 is None:
        lat0 = float(np.nanmedian(lat))
    R = 6371000.0
    x = np.deg2rad(lon) * cos(np.deg2rad(lat0)) * R
    y = np.deg2rad(lat) * R
    return x, y

def load_l2_flags_any(nc_path):
    for g in ("geophysical_data","navigation_data","geolocation_data", None):
        try:
            d = xr.open_dataset(nc_path, group=g) if g is not None else xr.open_dataset(nc_path)
            if "l2_flags" in d.variables:
                arr = np.asarray(d["l2_flags"].data); d.close(); return arr
            d.close()
        except Exception:
            pass
    return None

def load_scene(nc_path, group="geophysical_data", varname="chlor_a"):
    gphys = xr.open_dataset(nc_path, group=group)
    da = gphys[varname]
    chl = np.asarray(da.data, dtype=float)
    H, W = chl.shape

    flags = load_l2_flags_any(nc_path)
    if flags is None:
        land_mask = np.zeros_like(chl, dtype=bool)
    else:
        if flags.shape != chl.shape:
            if HAVE_SCIPY_NDIMAGE:
                zoom_factors = (chl.shape[0]/flags.shape[0], chl.shape[1]/flags.shape[1])
                flags = scipy_zoom(flags, zoom_factors, order=0, prefilter=False)
                flags = flags[:chl.shape[0], :chl.shape[1]]
            else:
                flags = None
        if flags is None:
            land_mask = np.zeros_like(chl, dtype=bool)
        else:
            LAND_BIT  = 1
            land_mask = (flags.astype(np.uint32) & (1 << LAND_BIT)) != 0

    have_geo = True
    lat = lon = None
    try:
        nav = xr.open_dataset(nc_path, group="navigation_data")
        lat = np.asarray(nav["latitude"].data, dtype=float)
        lon = np.asarray(nav["longitude"].data, dtype=float)
        nav.close()
    except Exception:
        pass
    if (lat is None) or (lon is None):
        try:
            geo = xr.open_dataset(nc_path, group="geolocation_data")
            lat = np.asarray(geo["latitude"].data, dtype=float)
            lon = np.asarray(geo["longitude"].data, dtype=float)
            geo.close()
        except Exception:
            have_geo = False

    if have_geo and (lat is not None) and (lon is not None):
        if lat.ndim == 1 and lon.ndim == 1:
            lon, lat = np.meshgrid(lon, lat, indexing='xy')
    else:
        yy, xx = np.indices((H, W))
        lat, lon = yy.astype(float), xx.astype(float)
        have_geo = False

    gphys.close()
    return chl, lat, lon, land_mask, have_geo

def compute_rectilinear_transform(lat, lon):
    """Return (Affine transform, crs) for near-rectilinear WGS84 grids, else (None,None)."""
    if not HAVE_RASTERIO:
        return None, None
    try:
        lat_row = lat[:, 0]; lon_col = lon[0, :]
        dlat = np.nanmedian(np.diff(lat_row)); dlon = np.nanmedian(np.diff(lon_col))
        if not np.isfinite(dlat) or not np.isfinite(dlon) or dlat == 0 or dlon == 0:
            return None, None
        lat0 = float(lat[0,0]); lon0 = float(lon[0,0])
        transform = Affine(dlon, 0, lon0, 0, dlat, lat0)
        crs = "EPSG:4326"
        return transform, crs
    except Exception:
        return None, None

def save_png(array, out_png, vmin=None, vmax=None, cmap='viridis'):
    plt.figure(figsize=(9,7))
    plt.imshow(np.array(array, dtype=float), origin='upper', vmin=vmin, vmax=vmax, cmap=cmap)
    plt.axis('off'); plt.tight_layout()
    plt.savefig(out_png, dpi=DPI, bbox_inches='tight', pad_inches=0.0)
    plt.close()

def save_geotiff(array, out_tif, transform, crs, nodata=np.nan, dtype=np.float32):
    if not HAVE_RASTERIO or transform is None or crs is None:
        return False
    arr = np.asarray(array, dtype=dtype)
    profile = {
        "driver": "GTiff", "height": arr.shape[0], "width": arr.shape[1],
        "count": 1, "dtype": str(arr.dtype), "crs": crs, "transform": transform,
        "compress": "deflate", "tiled": True
    }
    with rasterio.open(out_tif, "w", **profile) as dst:
        dst.write(arr, 1)
        if np.issubdtype(arr.dtype, np.floating):
            dst.update_tags(1, NODATA=nodata)
    return True


# ======================= SEAMLESS (GLOBAL) OK + IDW =======================
def estimate_variogram_params_subsample(x, y, z, max_points=5000, max_pairs=100_000, n_bins=18, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    valid = np.isfinite(z)
    idxs = np.column_stack(np.nonzero(valid))
    if idxs.shape[0] < 60:
        return 0.0, float(np.nanvar(z)), 2000.0
    n_samp = min(idxs.shape[0], max_points)
    samp = idxs[rng.choice(idxs.shape[0], size=n_samp, replace=False)]
    n_pairs = min(max_pairs, n_samp*(n_samp-1)//2)
    i = rng.integers(0, n_samp, size=n_pairs)
    j = rng.integers(0, n_samp, size=n_pairs)
    r1, c1 = samp[i,0], samp[i,1]; r2, c2 = samp[j,0], samp[j,1]
    d = np.sqrt((x[r1,c1]-x[r2,c2])**2 + (y[r1,c1]-y[r2,c2])**2)
    g = 0.5*(z[r1,c1]-z[r2,c2])**2
    if not np.isfinite(d).any() or np.nanmax(d) <= 0:
        return 0.0, float(np.nanvar(z)), 2000.0
    bins = np.linspace(0, np.nanpercentile(d,95), n_bins+1)
    bc = 0.5*(bins[:-1]+bins[1:])
    gam = np.full(n_bins, np.nan)
    for k in range(n_bins):
        m = (d>=bins[k]) & (d<bins[k+1])
        if np.any(m): gam[k] = np.nanmedian(g[m])
    nug = np.nanmedian(gam[:2]); sill = np.nanmedian(gam[-4:])
    if not np.isfinite(sill) or sill <= 0: sill = float(np.nanvar(z))
    if not np.isfinite(nug) or nug < 0:   nug = 0.0
    target = nug + 0.95*sill
    a = None
    for bci, gvi in zip(bc, gam):
        if np.isfinite(gvi) and gvi >= target: a = bci; break
    if a is None or a <= 0:
        a = np.nanmedian(bc[np.isfinite(gam)]) if np.any(np.isfinite(gam)) else 2000.0
    if not np.isfinite(a) or a <= 0: a = 2000.0
    return float(nug), float(sill), float(a)

def idw_points(x_obs, y_obs, z_obs, x_tgt, y_tgt, k=N_CLOSEST_POINTS, radius=MIN_RADIUS_M):
    if not HAVE_KDTREE:
        raise RuntimeError("IDW fallback requires scipy.spatial.cKDTree.")
    tree = cKDTree(np.column_stack([x_obs, y_obs]))
    z_hat = np.full(x_tgt.shape, np.nan, float)
    var   = np.full(x_tgt.shape, np.nan, float)
    for i in range(x_tgt.size):
        d, ii = tree.query([x_tgt[i], y_tgt[i]], k=k, distance_upper_bound=radius)
        m = np.isfinite(d) & (ii < x_obs.size)
        if not np.any(m): continue
        d = d[m]; v = z_obs[ii[m]]
        d[d == 0] = 1e-6
        w = 1.0/(d**2)
        z_hat[i] = np.sum(w*v)/np.sum(w)
        var[i]   = np.mean(d)/(np.std(d)+1e-6)
    return z_hat, var

def fill_seamless_ok(chl, lat, lon, land_mask):
    """Single global OK model (log10-space) with batched prediction; IDW fallback."""
    H, W = chl.shape

    have_geo = not (lat is None or lon is None)
    if have_geo:
        x, y = latlon_to_xy_m(lat, lon)
    else:
        yy, xx = np.indices((H, W))
        x, y = xx.astype(float), yy.astype(float)

    obs_mask = np.isfinite(chl) & (chl > 0)
    tgt_mask = (~np.isfinite(chl)) | (chl <= 0)
    if land_mask is not None and land_mask.shape == chl.shape:
        obs_mask &= ~land_mask
        tgt_mask &= ~land_mask

    logA = np.full_like(chl, np.nan, float)
    logA[obs_mask] = np.log10(chl[obs_mask])

    # Global variogram
    nug, sill, krange = estimate_variogram_params_subsample(x, y, logA)
    log(f"[Global variogram] exp: nugget={nug:.3g}, sill={sill:.3g}, range≈{int(krange)}")

    # Observations (thin if needed)
    oy, ox = np.nonzero(obs_mask)
    if oy.size == 0:
        raise RuntimeError("No observations found.")
    rng = np.random.default_rng(RANDOM_SEED)
    if oy.size > MAX_GLOBAL_OBS:
        sel = rng.choice(oy.size, size=MAX_GLOBAL_OBS, replace=False)
        oy, ox = oy[sel], ox[sel]
        log(f"[Obs] thinned → {oy.size:,}")
    else:
        log(f"[Obs] using {oy.size:,} points")

    xo, yo, zo = x[oy, ox], y[oy, ox], logA[oy, ox]

    # Targets
    ty, tx = np.nonzero(tgt_mask)
    n_tgt = ty.size
    log(f"[Targets] {n_tgt:,} pixels to fill")

    logA_out = logA.copy()
    var_map  = np.full_like(logA, np.nan, float)

    # Try OK once
    use_idw_all = not HAVE_PYKRIGE
    ok = None
    if not use_idw_all:
        try:
            ok = PKOrdinaryKriging(
                xo, yo, zo,
                variogram_model=VARIOGRAM_MODEL,
                variogram_parameters={'sill': sill, 'range': krange, 'nugget': nug},
                coordinates_type='euclidean'
            )
            log("[OK] model initialized")
        except (MemoryError, Exception) as e:
            log(f"[OK] init failed: {e} → falling back to IDW.")
            use_idw_all = True

    # Predict in batches
    if n_tgt > 0:
        for start in range(0, n_tgt, BATCH_TARGETS):
            stop = min(start + BATCH_TARGETS, n_tgt)
            idx = slice(start, stop)
            xt, yt = x[ty[idx], tx[idx]], y[ty[idx], tx[idx]]

            if not use_idw_all:
                try:
                    z_hat, ss = ok.execute(
                        style='points',
                        xpoints=xt, ypoints=yt,
                        n_closest_points=int(N_CLOSEST_POINTS),
                        backend='loop'
                    )
                    z_hat = np.asarray(z_hat).ravel()
                    ss    = np.asarray(ss).ravel()
                except (MemoryError, Exception) as e:
                    log(f"[OK] batch {start}:{stop} failed ({e}); switching this batch to IDW.")
                    z_hat, ss = idw_points(xo, yo, zo, xt, yt, k=int(N_CLOSEST_POINTS),
                                           radius=max(krange*2.0, MIN_RADIUS_M))
            else:
                z_hat, ss = idw_points(xo, yo, zo, xt, yt, k=int(N_CLOSEST_POINTS),
                                       radius=max(krange*2.0, MIN_RADIUS_M))

            logA_out[ty[idx], tx[idx]] = z_hat
            var_map [ty[idx], tx[idx]] = ss

            if stop == n_tgt:
                log(f"  filled {stop:,}/{n_tgt:,}")

    # Back-transform & masks
    chl_before = chl.copy()
    chl_after  = np.full_like(chl, np.nan, float)
    m = np.isfinite(logA_out)
    chl_after[m] = 10**logA_out[m]

    where_filled = np.isfinite(chl_after) & (~np.isfinite(chl_before) | (chl_before <= 0))
    if land_mask is not None and land_mask.shape == chl.shape:
        where_filled &= ~land_mask

    # Confidence (invert variance across filled pixels; normalize 5–95%)
    conf = np.full_like(var_map, np.nan, float)
    fv = np.isfinite(var_map) & where_filled
    if np.any(fv):
        vmin = np.nanpercentile(var_map[fv], 5)
        vmax = np.nanpercentile(var_map[fv], 95)
        if vmax > vmin:
            conf[fv] = 1.0 - (var_map[fv] - vmin) / (vmax - vmin)
            conf[fv] = np.clip(conf[fv], 0, 1)
        else:
            conf[fv] = 1.0

    return chl_before, chl_after, where_filled, conf


# ======================= DIAGNOSTICS =======================
def _nearest_rc_for_latlon(TLAT, TLON, LAT, LON):
    X, Y = latlon_to_xy_m(LAT, LON)
    R = 6371000.0
    xt = np.deg2rad(TLON) * cos(np.deg2rad(float(np.nanmedian(LAT)))) * R
    yt = np.deg2rad(TLAT) * R
    d2 = (X - xt)**2 + (Y - yt)**2
    return np.unravel_index(int(np.nanargmin(d2)), LAT.shape)

def _summ(vals, mask):
    n = int(np.count_nonzero(mask))
    if n == 0:
        return {"mean": None, "median": None, "variance": None, "valid": 0, "total": int(vals.size)}
    return {
        "mean":     float(np.nanmean(vals[mask])),
        "median":   float(np.nanmedian(vals[mask])),
        "variance": float(np.nanvar(vals[mask])),
        "valid":    n,
        "total":    int(vals.size),
    }

def _window_bounds(r, c, size, H, W):
    half = (size - 1) // 2
    r0 = max(0, r - half); r1 = min(H, r + half + 1)
    c0 = max(0, c - half); c1 = min(W, c + half + 1)
    return r0, r1, c0, c1

def _window_stats(val_arr, conf_arr, r, c, size, land=None):
    H, W = val_arr.shape
    r0, r1, c0, c1 = _window_bounds(r, c, size, H, W)
    win_val  = val_arr[r0:r1, c0:c1]
    win_conf = conf_arr[r0:r1, c0:c1] if (conf_arr is not None and conf_arr.shape == val_arr.shape) else None
    if land is not None and isinstance(land, np.ndarray) and land.shape == val_arr.shape:
        land_win = land[r0:r1, c0:c1]
        win_val  = np.where(land_win, np.nan, win_val)
        if win_conf is not None:
            win_conf = np.where(land_win, np.nan, win_conf)
    m_val  = np.isfinite(win_val)
    out = {"chl": _summ(win_val, m_val), "r0": r0, "r1": r1, "c0": c0, "c1": c1}
    if win_conf is not None:
        m_conf = np.isfinite(win_conf)
        out["confidence"] = _summ(win_conf, m_conf)
    return out

def _nonkrige_mean(arr, land, r, c, size):
    H, W = arr.shape
    r0, r1, c0, c1 = _window_bounds(r, c, size, H, W)
    win = arr[r0:r1, c0:c1]
    m = np.isfinite(win) & (win > 0) & (~land[r0:r1, c0:c1])
    return float(np.nanmean(win[m])) if np.any(m) else None

def compute_diagnostics(chl_before, chl_after, conf, land_mask, lat, lon,
                        insitu_value=None, target_lat=None, target_lon=None, csv_row=None):
    H, W = chl_after.shape
    if (target_lat is not None) and (target_lon is not None):
        rr, cc = _nearest_rc_for_latlon(target_lat, target_lon, lat, lon)
    elif csv_row is not None and pd.notna(csv_row.get("nearest_row")) and pd.notna(csv_row.get("nearest_col")):
        rr, cc = int(csv_row.get("nearest_row")), int(csv_row.get("nearest_col"))
        rr = min(max(0, rr), H-1); cc = min(max(0, cc), W-1)
    else:
        rr = H//2; cc = W//2

    # Early skip if the original 10×10 has no valid water
    r0s, r1s, c0s, c1s = _window_bounds(rr, cc, 10, H, W)
    orig10 = chl_before[r0s:r1s, c0s:c1s]
    orig10_valid = np.isfinite(orig10) & (orig10 > 0) & (~land_mask[r0s:r1s, c0s:c1s])
    if int(np.count_nonzero(orig10_valid)) == 0:
        return {"skip": True, "reason": "No valid (>0) water pixels in original 10x10 neighborhood."}

    center_val  = float(chl_after[rr, cc]) if np.isfinite(chl_after[rr, cc]) else None
    center_conf = float(conf[rr, cc]) if (conf is not None and np.isfinite(conf[rr, cc])) else None
    on_land     = bool(land_mask[rr, cc])

    s3  = _window_stats(chl_after, conf, rr, cc, 3,  land_mask)
    s5  = _window_stats(chl_after, conf, rr, cc, 5,  land_mask)
    s10 = _window_stats(chl_after, conf, rr, cc, 11, land_mask)

    # Envelope & robust z on 10x10
    r0, r1, c0, c1 = s10["r0"], s10["r1"], s10["c0"], s10["c1"]
    win10 = chl_after[r0:r1, c0:c1]
    m10   = np.isfinite(win10)
    local_q10 = float(np.nanpercentile(win10[m10], 10)) if np.any(m10) else None
    local_q90 = float(np.nanpercentile(win10[m10], 90)) if np.any(m10) else None
    within_envelope = None
    if (center_val is not None) and (local_q10 is not None) and (local_q90 is not None):
        within_envelope = bool((center_val >= local_q10) and (center_val <= local_q90))
    mad = float(np.nanmedian(np.abs(win10[m10] - np.nanmedian(win10[m10])))) if np.any(m10) else None
    robust_z = None
    if (mad is not None) and (mad > 0) and (center_val is not None):
        robust_z = float((center_val - np.nanmedian(win10[m10])) / (1.4826*mad))
    IQR_10 = float(np.nanpercentile(win10[m10], 75) - np.nanpercentile(win10[m10], 25)) if np.any(m10) else None

    nearest_obs_dist = None
    if HAVE_KDTREE:
        try:
            x, y = latlon_to_xy_m(lat, lon)
            oy, ox = np.nonzero(np.isfinite(chl_before) & (chl_before > 0) & (~land_mask))
            if oy.size > 0:
                xo, yo = x[oy, ox], y[oy, ox]
                tree = cKDTree(np.column_stack([xo.ravel(), yo.ravel()]))
                d, _ = tree.query([x[rr,cc], y[rr,cc]], k=1)
                nearest_obs_dist = float(d)
        except Exception:
            pass

    # non-krige mean (orig) for |k−nk|
    nk3 = _nonkrige_mean(chl_before, land_mask, rr, cc, 3)
    k3_mean = s3["chl"]["mean"]
    abs_k_vs_nk = (abs(k3_mean - nk3) if (k3_mean is not None and nk3 is not None) else None)
    abs_k_vs_is = (abs(k3_mean - insitu_value) if (k3_mean is not None and (insitu_value is not None)) else None)

    return {
        "skip": False,
        "row": rr, "col": cc,
        "center_value_mg_m3": center_val,
        "center_confidence": center_conf,
        "on_land": on_land,
        "stats": {"3x3": s3, "5x5": s5, "10x10": s10},
        "local_q10": local_q10, "local_q90": local_q90,
        "within_envelope": within_envelope,
        "IQR_10x10": IQR_10,
        "robust_z_10x10": robust_z,
        "nearest_obs_dist": nearest_obs_dist, "nearest_obs_dist_units": "m",
        "krige3x3_mean": k3_mean,
        "nonkrige3x3_mean": nk3,
        "abs_k_vs_nk": abs_k_vs_nk,
        "abs_k_vs_is": abs_k_vs_is,
    }


# ======================= NC WRITER (COPY INTO OUTPUT_DIR) =======================
def write_seamless_nc_copy(src_nc, out_nc_path, A, WF, CF,
                           group="geophysical_data", src_var="chlor_a"):
    if not HAVE_NETCDF4:
        raise RuntimeError("netCDF4 is required to write output.")

    H, W = A.shape
    out_nc_path = str(out_nc_path)
    shutil.copy2(src_nc, out_nc_path)

    def _get_dims(nc_path, group="geophysical_data", varname="chlor_a"):
        try:
            with Dataset(nc_path, 'r') as nc:
                if group in nc.groups and varname in nc.groups[group].variables:
                    dims = nc.groups[group].variables[varname].dimensions
                    if len(dims) >= 2:
                        return dims[-2], dims[-1]
        except Exception:
            pass
        return "number_of_lines", "pixels_per_line"

    def _copy_attrs_from(nc_group, src_name):
        attrs = {}
        if src_name in nc_group.variables:
            vsrc = nc_group.variables[src_name]
            for k in vsrc.ncattrs():
                if k != "_FillValue":
                    try:
                        attrs[k] = getattr(vsrc, k)
                    except Exception:
                        pass
        return attrs

    def _upsert(g, name, dtype, data, dims, attrs=None, fill_val=None, zlib=True, complevel=4):
        if name in g.variables:
            v = g.variables[name]
            if v.shape != data.shape:
                raise ValueError(f"Existing var '{name}' has shape {v.shape}, expected {data.shape}.")
            v[:] = data
        else:
            kw = {}
            try:
                kw = {"zlib": zlib, "complevel": complevel}
                if fill_val is not None:
                    kw["_FillValue"] = dtype(fill_val) if np.issubdtype(dtype, np.floating) else fill_val
                v = g.createVariable(name, dtype, dimensions=dims, **kw)
            except TypeError:
                v = g.createVariable(name, dtype, dimensions=dims)
                if fill_val is not None:
                    try:
                        setattr(v, "_FillValue", dtype(fill_val))
                    except Exception:
                        pass
        v[:] = data
        if attrs:
            for k, val in attrs.items():
                try:
                    setattr(v, k, val)
                except Exception:
                    pass

    with Dataset(out_nc_path, 'r+') as nc:
        gname = group
        g = nc.groups[gname] if gname in nc.groups else nc.createGroup(gname)
        yd, xd = _get_dims(out_nc_path, gname, src_var)

        if yd not in nc.dimensions: nc.createDimension(yd, H)
        if xd not in nc.dimensions: nc.createDimension(xd, W)
        dims = (yd, xd)

        base_attrs = _copy_attrs_from(g, src_var)
        base_attrs.update({
            "long_name": "Chlorophyll Concentration, gap-filled (OK/IDW, seamless global)",
            "comment": "Single global OK (log10) with IDW fallback; LAND excluded.",
            "source_variable": src_var, "units": "mg m-3",
        })
        wf_attrs = {"long_name": "Locations filled by interpolation",
                    "flag_values": "0,1", "flag_meanings": "not_filled filled"}
        cf_attrs = {"long_name": "Confidence (inverted/normalized kriging variance or IDW proxy)",
                    "valid_min": 0.0, "valid_max": 1.0,
                    "comment": "1=high; NaN outside filled water pixels."}

        _upsert(g, "chlor_a_filled", np.float32, np.asarray(A,  dtype=np.float32), dims, base_attrs, fill_val=np.nan)
        _upsert(g, "where_filled",   np.uint8,  np.asarray(WF, dtype=np.uint8),   dims, wf_attrs,   fill_val=0)
        if CF is not None:
            _upsert(g, "confidence", np.float32, np.asarray(CF, dtype=np.float32),   dims, cf_attrs,   fill_val=np.nan)

        stamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
        hist = getattr(nc, "history", "")
        line = f"{stamp} : added geophysical_data/chlor_a_filled, where_filled, confidence (seamless)."
        nc.history = (hist + "\n" + line).strip()

    return out_nc_path


# ======================= MAIN (BATCH ALL ROWS) =======================
def main():
    if not SUMMARY_CSV.exists():
        raise FileNotFoundError(f"Summary CSV not found: {SUMMARY_CSV}")

    ensure_output_dir()

    df = pd.read_csv(SUMMARY_CSV, dtype={"file": "string"})
    if "file" not in df.columns:
        raise RuntimeError("Summary CSV must include a 'file' column with .nc paths.")
    has_insitu = ("in-situ chl-a" in df.columns)

    rows_out = []

    for i, row in df.iterrows():
        raw_nc_path = str(row["file"] or "").strip()
        if not raw_nc_path:
            log(f"[{i}] Empty 'file' — skip.")
            continue

        insitu_value = None
        if has_insitu:
            try:
                v = row["in-situ chl-a"]
                insitu_value = float(v) if pd.notna(v) else None
            except Exception:
                insitu_value = None

        nc_path = remap_windows_path_to_local(raw_nc_path)
        log(f"[{i}] Input: {raw_nc_path}")
        log(f"[{i}] Local: {nc_path}")

        if not nc_path.exists():
            log(f"[{i}] MISSING on local FS — skip.")
            rows_out.append({"row_index_1based": i+1, "file_raw": raw_nc_path, "file_local": str(nc_path), "error": "missing file"})
            continue

        base = Path(nc_path).stem
        try:
            # Load scene
            chl, lat, lon, land_mask, have_geo = load_scene(str(nc_path), group=GROUP, varname=VARNAME)
            H, W = chl.shape
            land_pct = 100.0 * (np.sum(land_mask) / (H*W)) if land_mask is not None else 0.0
            log(f"[{i}] Scene: {H}x{W}, geo={have_geo}, land≈{land_pct:.1f}%")

            # Seamless/global OK + IDW
            chl_before, chl_after, where_filled, conf = fill_seamless_ok(chl, lat, lon, land_mask)
            n_filled = int(np.sum(where_filled))
            log(f"[{i}] Newly filled pixels: {n_filled}")

            # Save PNGs
            out_png_orig  = OUTPUT_DIR / f"{base}_orig_masked.png"
            out_png_fill  = OUTPUT_DIR / f"{base}_filled.png"
            out_png_where = OUTPUT_DIR / f"{base}_where_filled.png"
            out_png_conf  = OUTPUT_DIR / f"{base}_confidence.png"

            save_png(np.where(land_mask, np.nan, chl_before), out_png_orig,  vmin=CHL_VMIN, vmax=CHL_VMAX, cmap='viridis')
            save_png(np.where(land_mask, np.nan, chl_after),  out_png_fill,  vmin=CHL_VMIN, vmax=CHL_VMAX, cmap='viridis')
            save_png(where_filled.astype(float),              out_png_where, vmin=0,        vmax=1,        cmap='gray')
            save_png(conf,                                     out_png_conf, vmin=0,        vmax=1,        cmap='viridis')

            # Save GeoTIFFs (if rectilinear)
            if HAVE_RASTERIO:
                transform, crs = compute_rectilinear_transform(lat, lon)
                if transform is not None and crs is not None:
                    out_tif_orig  = OUTPUT_DIR / f"{base}_orig_masked.tif"
                    out_tif_fill  = OUTPUT_DIR / f"{base}_filled.tif"
                    out_tif_where = OUTPUT_DIR / f"{base}_where_filled.tif"
                    out_tif_conf  = OUTPUT_DIR / f"{base}_confidence.tif"

                    save_geotiff(np.where(land_mask, np.nan, chl_before).astype(np.float32), out_tif_orig,  transform, crs)
                    save_geotiff(np.where(land_mask, np.nan, chl_after ).astype(np.float32), out_tif_fill,  transform, crs)
                    save_geotiff(where_filled.astype(np.uint8),                              out_tif_where, transform, crs, nodata=0, dtype=np.uint8)
                    save_geotiff(np.asarray(conf, dtype=np.float32),                         out_tif_conf,  transform, crs)
                else:
                    log(f"[{i}] GeoTIFF skipped (non-rectilinear grid or no rasterio).")

            # Write _seamless.nc copy into OUTPUT_DIR
            out_nc = None
            if HAVE_NETCDF4:
                out_nc = OUTPUT_DIR / f"{base}_seamless.nc"
                write_seamless_nc_copy(str(nc_path), out_nc, chl_after, where_filled, conf, group=GROUP, src_var=VARNAME)
                log(f"[{i}] seamless.nc written: {out_nc}")
            else:
                log(f"[{i}] netCDF4 not available; seamless.nc skipped.")

            # Diagnostics at target
            diag = compute_diagnostics(
                chl_before, chl_after, conf, land_mask, lat, lon,
                insitu_value=insitu_value,
                target_lat=TARGET_LAT, target_lon=TARGET_LON, csv_row=None
            )

            # Flatten to CSV row
            def _flat(prefix, d):
                out = {}
                if d is None: return out
                for k, v in d.items():
                    if isinstance(v, dict):
                        for k2, v2 in v.items():
                            out[f"{prefix}.{k}.{k2}"] = v2
                    else:
                        out[f"{prefix}.{k}"] = v
                return out

            if diag.get("skip", False):
                rec = {
                    "row_index_1based": i+1,
                    "file_raw": raw_nc_path,
                    "file_local": str(nc_path),
                    "skip": True,
                    "skip_reason": diag.get("reason"),
                    "scene_H": H, "scene_W": W, "have_geo": bool(have_geo), "land_pct": land_pct,
                    "filled_pixels": n_filled,
                    "outputs_dir": str(OUTPUT_DIR),
                    "out_nc": str(out_nc) if out_nc else "",
                }
            else:
                rec = {
                    "row_index_1based": i+1,
                    "file_raw": raw_nc_path,
                    "file_local": str(nc_path),
                    "scene_H": H, "scene_W": W,
                    "have_geo": bool(have_geo),
                    "land_pct": land_pct,
                    "filled_pixels": n_filled,
                    "outputs_dir": str(OUTPUT_DIR),
                    "out_nc": str(out_nc) if out_nc else "",
                    "target_row": diag["row"],
                    "target_col": diag["col"],
                    "center_value_mg_m3": diag["center_value_mg_m3"],
                    "center_confidence": diag["center_confidence"],
                    "on_land": diag["on_land"],
                    "local_q10": diag["local_q10"],
                    "local_q90": diag["local_q90"],
                    "within_envelope": diag["within_envelope"],
                    "IQR_10x10": diag["IQR_10x10"],
                    "robust_z_10x10": diag["robust_z_10x10"],
                    "nearest_obs_dist_m": diag["nearest_obs_dist"],
                    "krige3x3_mean": diag["krige3x3_mean"],
                    "krige3x3_median":   diag["stats"]["3x3"]["chl"]["median"],
                    "krige5x5_mean":     diag["stats"]["5x5"]["chl"]["mean"],
                    "krige5x5_median":   diag["stats"]["5x5"]["chl"]["median"],
                    "krige10x10_mean":   diag["stats"]["10x10"]["chl"]["mean"],
                    "krige10x10_median": diag["stats"]["10x10"]["chl"]["median"],
                    "insitu_value": insitu_value,
                    "abs_k_vs_nk": diag["abs_k_vs_nk"],
                    "abs_k_vs_is": diag["abs_k_vs_is"],
                }
                # Optional: flatten full stats blocks too
                for wnd in ("3x3","5x5","10x10"):
                    ws = diag["stats"].get(wnd, {})
                    rec.update(_flat(f"stats.{wnd}", ws))

            rows_out.append(rec)

        except Exception as e:
            log(f"[{i}] ERROR: {Path(nc_path).name} → {e}")
            rows_out.append({
                "row_index_1based": i+1,
                "file_raw": raw_nc_path,
                "file_local": str(nc_path),
                "outputs_dir": str(OUTPUT_DIR),
                "error": str(e)
            })

    # Save CSV in OUTPUT_DIR
    out_csv = OUTPUT_DIR / f"{SITE}_krige_stats_v3.csv"
    pd.DataFrame(rows_out).to_csv(out_csv, index=False)
    log(f"CSV written: {out_csv}")

if __name__ == "__main__":
    main()


[09:04:22] [0] Input: Y:\Mingyue\LISCO\l2gen-matchup-product\S2A_MSIL1C_20150802T155136_N0500_R011_T18TXL_20231016T221500.L2.nc
[09:04:22] [0] Local: /Volumes/purkislab2a/Mingyue/LISCO/l2gen-matchup-product/S2A_MSIL1C_20150802T155136_N0500_R011_T18TXL_20231016T221500.L2.nc
[09:05:09] [0] Scene: 5490x5490, geo=True, land≈61.3%
[09:05:10] [Global variogram] exp: nugget=0.0314, sill=0.115, range≈44536
[09:05:10] [Obs] thinned → 120,000
[09:05:10] [Targets] 1,622,350 pixels to fill
[09:06:02]   filled 1,622,350/1,622,350
[09:06:03] [0] Newly filled pixels: 1622350


/var/folders/77/yb_ktv2d4vlfw_lrrj_xl8f40000gn/T/ipykernel_31061/1134916802.py:618: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


[09:07:01] [0] seamless.nc written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/S2A_MSIL1C_20150802T155136_N0500_R011_T18TXL_20231016T221500.L2_seamless.nc
[09:07:06] [1] Input: Y:\Mingyue\LISCO\l2gen-matchup-product\S2A_MSIL1C_20150901T155136_N0500_R011_T18TXL_20231017T091839.L2.nc
[09:07:06] [1] Local: /Volumes/purkislab2a/Mingyue/LISCO/l2gen-matchup-product/S2A_MSIL1C_20150901T155136_N0500_R011_T18TXL_20231017T091839.L2.nc
[09:07:11] [1] Scene: 5490x5490, geo=True, land≈61.3%
[09:07:12] [Global variogram] exp: nugget=0.0171, sill=0.112, range≈44062
[09:07:12] [Obs] thinned → 120,000
[09:07:12] [Targets] 842,494 pixels to fill
[09:07:38]   filled 842,494/842,494
[09:07:39] [1] Newly filled pixels: 842494


/var/folders/77/yb_ktv2d4vlfw_lrrj_xl8f40000gn/T/ipykernel_31061/1134916802.py:618: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


[09:08:36] [1] seamless.nc written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/S2A_MSIL1C_20150901T155136_N0500_R011_T18TXL_20231017T091839.L2_seamless.nc
[09:08:41] [2] Input: Y:\Mingyue\LISCO\l2gen-matchup-product\S2A_MSIL1C_20151021T155022_N0500_R011_T18TXL_20231017T042402.L2.nc
[09:08:41] [2] Local: /Volumes/purkislab2a/Mingyue/LISCO/l2gen-matchup-product/S2A_MSIL1C_20151021T155022_N0500_R011_T18TXL_20231017T042402.L2.nc
[09:08:46] [2] Scene: 5490x5490, geo=True, land≈61.3%
[09:08:46] [Global variogram] exp: nugget=0.0184, sill=0.0437, range≈43709
[09:08:46] [Obs] thinned → 120,000
[09:08:47] [Targets] 966,549 pixels to fill
[09:09:16]   filled 966,549/966,549
[09:09:17] [2] Newly filled pixels: 966549


...

[12:07:23] [66] seamless.nc written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/S2A_MSIL1C_20210611T153911_N0500_R011_T18TXL_20230603T125252.L2_seamless.nc
[12:07:24] [67] Input: Y:\Mingyue\LISCO\l2gen-matchup-product\S2B_MSIL1C_20210716T153809_N0500_R011_T18TXL_20230130T163211.L2.nc
[12:07:24] [67] Local: /Volumes/purkislab2a/Mingyue/LISCO/l2gen-matchup-product/S2B_MSIL1C_20210716T153809_N0500_R011_T18TXL_20230130T163211.L2.nc
[12:07:28] [67] Scene: 5490x5490, geo=True, land≈61.3%
[12:07:29] [Global variogram] exp: nugget=0.00197, sill=0.0184, range≈84643
[12:07:29] [Obs] thinned → 120,000
[12:07:29] [Targets] 4,755,494 pixels to fill
[12:09:48]   filled 4,755,494/4,755,494
[12:09:49] [67] Newly filled pixels: 4755494


/var/folders/77/yb_ktv2d4vlfw_lrrj_xl8f40000gn/T/ipykernel_31061/1134916802.py:618: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


[12:10:50] [67] seamless.nc written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/S2B_MSIL1C_20210716T153809_N0500_R011_T18TXL_20230130T163211.L2_seamless.nc
[12:10:54] [68] Input: Y:\Mingyue\LISCO\l2gen-matchup-product\S2A_MSIL1C_20210731T153911_N0500_R011_T18TXL_20230603T102903.L2.nc
[12:10:54] [68] Local: /Volumes/purkislab2a/Mingyue/LISCO/l2gen-matchup-product/S2A_MSIL1C_20210731T153911_N0500_R011_T18TXL_20230603T102903.L2.nc
[12:11:00] [68] Scene: 5490x5490, geo=True, land≈61.3%
[12:11:01] [Global variogram] exp: nugget=0.042, sill=0.152, range≈39704
[12:11:01] [Obs] thinned → 120,000
[12:11:01] [Targets] 5,485,748 pixels to fill
[12:13:43]   filled 5,485,748/5,485,748
[12:13:44] [68] Newly filled pixels: 5485748


/var/folders/77/yb_ktv2d4vlfw_lrrj_xl8f40000gn/T/ipykernel_31061/1134916802.py:618: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


[12:14:40] [68] seamless.nc written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/S2A_MSIL1C_20210731T153911_N0500_R011_T18TXL_20230603T102903.L2_seamless.nc
[12:14:43] CSV written: /Volumes/purkislab2a/Mingyue/LISCO/krige_outputs_3/LISCO_krige_stats_v3.csv
